<a href="https://colab.research.google.com/github/siddhantdubey/GenderedMovieDialogue/blob/master/GenderClassifierLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/Folders/Work/Research/CornellMovie/data/"


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
import torchtext 
import numpy as np
import pandas as pd
from torchtext import data
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence 
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
X_train = []
Y_train = []
X_test = []
Y_test = []

In [4]:
with open(root_dir + 'training_set.txt', encoding="charmap") as training:
  for line in training:
    line = line.strip()
    line_num, chr_id, movie_id, chr_name, chr_gender, line_text, credit = line.split("+++$+++")
    if(chr_gender.strip().lower() == "m"):
      Y_train.append(0)
      X_train.append(line_text.strip())
    elif(chr_gender.strip().lower() == "f"):
      Y_train.append(1)
      X_train.append(line_text.strip())

with open(root_dir + 'test_set.txt', encoding="charmap") as test:
  for line in test:
    line = line.strip()
    line_num, chr_id, movie_id, chr_name, chr_gender, line_text, credit = line.split("+++$+++")
    if(chr_gender.strip().lower() == "m"):
      Y_test.append(0)
      X_test.append(line_text.strip())

    elif(chr_gender.strip().lower() == "f"):
      Y_test.append(1)
      X_test.append(line_text.strip())


In [5]:
count_vect = CountVectorizer()

X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [6]:
X_test_counts = count_vect.fit_transform(X_test)
X_test_tfidf = tfidf_transformer.fit_transform(X_test_counts)

In [7]:
device = torch.device("cuda")

In [8]:
df = pd.DataFrame()

In [9]:
indices = []
for i in range(len(X_train)):
  if(len(X_train[i]) > 0):
    answer = True 
  else:
    indices.append(i)

for index in sorted(indices, reverse=True):
    del X_train[index]
    del Y_train[index]


In [10]:
y_indexes = []
for i in range(len(X_test)):
  if(len(X_test[i]) > 0):
    answer = True
  else:
    y_indexes.append(i)

for index in sorted(y_indexes, reverse=True):
  del X_test[index]
  del Y_test[index]

In [12]:
X = X_train + X_test
Y = Y_train + Y_test
df['text'] = X_train 
df['target'] = Y_train 

In [66]:
Y.count(1)

65222

In [15]:
def get_top_data(top_n = 30000):
  top_data_df_male = df[df['target'] == 0].head(top_n)
  top_data_df_female = df[df['target'] == 1].head(top_n)
  data_df_small = pd.concat([top_data_df_male, top_data_df_female])
  return data_df_small

In [67]:
top_data_df_small = get_top_data(top_n=65222)

In [68]:
from sklearn.model_selection import train_test_split
def split_train_test(top_data_df_small, test_size=0.2, shuffle_state=True):
    X_train, X_test, Y_train, Y_test = train_test_split(top_data_df_small[['text']], 
                                                        top_data_df_small['target'], 
                                                        shuffle=shuffle_state,
                                                        test_size=test_size, 
                                                        random_state=15)
    print("Value counts for Train genders")
    print(Y_train.value_counts())
    print("Value counts for Test genders")
    print(Y_test.value_counts())
    print(type(X_train))
    print(type(Y_train))
    X_train = X_train.reset_index()
    X_test = X_test.reset_index()
    Y_train = Y_train.to_frame()
    Y_train = Y_train.reset_index()
    Y_test = Y_test.to_frame()
    Y_test = Y_test.reset_index()
    print(X_train.head())
    return X_train, X_test, Y_train, Y_test

# Call the train_test_split
X_train, X_test, Y_train, Y_test = split_train_test(top_data_df_small)

Value counts for Train genders
0    52115
1    46123
Name: target, dtype: int64
Value counts for Test genders
0    13107
1    11453
Name: target, dtype: int64
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>
    index                      text
0   48254                Thank you.
1  178412               That light.
2   87168              And why not?
3   12166  I need your help, Joyce.
4   42287          Yes, I knowbut


In [69]:
test_df = pd.DataFrame()

In [70]:
print(Y_train)

        index  target
0       48254       1
1      178412       1
2       87168       0
3       12166       0
4       42287       1
...       ...     ...
98233   68030       0
98234   12551       1
98235   10470       0
98236   73927       0
98237    9977       0

[98238 rows x 2 columns]


In [71]:
test_df['text'] = X_test['text']
test_df['target'] = Y_test['target']

In [72]:
test_df.head(10)

,text,target
0,Down there. Did you ever loose track of him?,0
1,"Don't go looking for it, Dil.",1
2,And General Savitzky.,0
3,You're such a kidder. I have a big surprise fo...,1
4,Which of you did it? Which of you made me the ...,1
5,I didn't know you were into that sort of thing.,1
6,"Oooh. Sassy. What do you care who feels me up,...",1
7,Can I?,0
8,No.,0
9,"I'm all ears, good buddy.",0


In [73]:
train_df = pd.DataFrame()
train_df['text'] = X_train['text']
train_df['target'] = Y_train['target']

In [74]:
train_df.head()

,text,target
0,Thank you.,1
1,That light.,1
2,And why not?,0
3,"I need your help, Joyce.",0
4,"Yes, I knowbut",1


In [75]:
SEED = 42

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [76]:
TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

In [77]:
class DataFrameDataset(data.Dataset):

    def __init__(self, df, fields, is_test=False, **kwargs):
        examples = []
        for i, row in df.iterrows():
            label = row.target if not is_test else None
            text = row.text
            examples.append(data.Example.fromlist([text, label], fields))

        super().__init__(examples, fields, **kwargs)

    @staticmethod
    def sort_key(ex):
        return len(ex.text)

    @classmethod
    def splits(cls, fields, train_df, val_df=None, test_df=None, **kwargs):
        train_data, val_data, test_data = (None, None, None)
        data_field = fields

        if train_df is not None:
            train_data = cls(train_df.copy(), data_field, **kwargs)
        if val_df is not None:
            val_data = cls(val_df.copy(), data_field, **kwargs)
        if test_df is not None:
            test_data = cls(test_df.copy(), data_field, True, **kwargs)

        return tuple(d for d in (train_data, val_data, test_data) if d is not None)

In [78]:
fields = [('text',TEXT), ('label',LABEL)]
train_ds, test_ds = DataFrameDataset.splits(fields, train_df=train_df, val_df=test_df)


In [79]:
MAX_VOCAB_SIZE = len(X_train)

TEXT.build_vocab(train_ds, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = 'glove.6B.200d',
                 unk_init = torch.Tensor.zero_)

In [80]:
LABEL.build_vocab(train_ds)

In [81]:
BATCH_SIZE = 256

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator = data.BucketIterator.splits(
    (train_ds, test_ds), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device)

In [82]:
# Hyperparameters
num_epochs = 25
learning_rate = 0.001

INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 200
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 4
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token] # padding

In [83]:
class LSTM_net(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        self.fc1 = nn.Linear(hidden_dim * 2, hidden_dim)
        
        self.fc2 = nn.Linear(hidden_dim, 1)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        
        # text = [sent len, batch size]
        
        embedded = self.embedding(text)
        
        # embedded = [sent len, batch size, emb dim]
        
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        
        #unpack sequence
        # output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        # output = [sent len, batch size, hid dim * num directions]
        # output over padding tokens are zero tensors
        
        # hidden = [num layers * num directions, batch size, hid dim]
        # cell = [num layers * num directions, batch size, hid dim]
        
        # concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        # and apply dropout
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        output = self.fc1(hidden)
        output = self.dropout(self.fc2(output))
                
        #hidden = [batch size, hid dim * num directions]
            
        return output

In [84]:
#creating instance of our LSTM_net class

model = LSTM_net(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

In [85]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)
model.embedding.weight.data.copy_(pretrained_embeddings)

torch.Size([36675, 200])


tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.1229,  0.5804, -0.0696,  ..., -0.0392, -0.1624, -0.0967],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.1321,  0.1112,  0.4337,  ...,  0.1854,  0.3073,  0.4766],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])

In [86]:
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.1229,  0.5804, -0.0696,  ..., -0.0392, -0.1624, -0.0967],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.1321,  0.1112,  0.4337,  ...,  0.1854,  0.3073,  0.4766],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])


In [87]:
model.to(device) #CNN to GPU


# Loss and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), learning_rate)


In [88]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [89]:
def train(model, iterator):
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        text, text_lengths = batch.text
        # for a in text_lengths:
        #   if(a.item <= 0):
        #     text_lengths.remove
        optimizer.zero_grad()
        predictions = model(text, text_lengths).squeeze(1)
        loss = criterion(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)

        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [90]:
def evaluate(model, iterator):
    
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            text, text_lengths = batch.text

            predictions = model(text, text_lengths).squeeze(1)
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)
            
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [91]:
import time

t = time.time()
loss=[]
acc=[]
val_acc=[]
val_losses=[]

for epoch in range(num_epochs):
    train_loss, train_acc = train(model, train_iterator)
    val_loss, valid_acc = evaluate(model, valid_iterator)
    print("Epoch " + str(epoch) + " :")
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\tVal Loss: {val_loss:.3f} | Val. Acc: {valid_acc*100:.2f}%')
    print('\n')
    loss.append(train_loss)
    acc.append(train_acc)
    val_acc.append(valid_acc)
    val_losses.append(val_loss)
print(f'time:{time.time()-t:.3f}')

Epoch 0 :
	Train Loss: 0.689 | Train Acc: 54.22%
	Val Loss: 0.672 | Val. Acc: 60.05%


Epoch 1 :
	Train Loss: 0.668 | Train Acc: 57.46%
	Val Loss: 0.649 | Val. Acc: 61.14%


Epoch 2 :
	Train Loss: 0.645 | Train Acc: 59.57%
	Val Loss: 0.645 | Val. Acc: 61.61%


Epoch 3 :
	Train Loss: 0.628 | Train Acc: 60.61%
	Val Loss: 0.632 | Val. Acc: 62.31%


Epoch 4 :
	Train Loss: 0.613 | Train Acc: 61.80%
	Val Loss: 0.637 | Val. Acc: 62.12%


Epoch 5 :
	Train Loss: 0.602 | Train Acc: 62.51%
	Val Loss: 0.639 | Val. Acc: 62.25%


Epoch 6 :
	Train Loss: 0.590 | Train Acc: 63.06%
	Val Loss: 0.640 | Val. Acc: 61.59%


Epoch 7 :
	Train Loss: 0.581 | Train Acc: 63.35%
	Val Loss: 0.649 | Val. Acc: 62.05%


Epoch 8 :
	Train Loss: 0.569 | Train Acc: 64.59%
	Val Loss: 0.662 | Val. Acc: 61.43%


Epoch 9 :
	Train Loss: 0.562 | Train Acc: 64.86%
	Val Loss: 0.665 | Val. Acc: 61.34%


Epoch 10 :
	Train Loss: 0.554 | Train Acc: 64.99%
	Val Loss: 0.695 | Val. Acc: 61.39%


Epoch 11 :
	Train Loss: 0.543 | Train Acc: